# Import necessary libraries

In [ ]:
import os
import string
from collections import Counter

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import seaborn as sns
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud

# Download NLTK data files

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

# Settings for visualizations

In [ ]:
%matplotlib inline
plt.style.use('ggplot')

# Load dataset

In [ ]:
dataset = load_dataset("ajaykarthick/imdb-movie-reviews")
dataset

## First glance at the dataset

Dataset is pretty big, so I will load only test data for analysis.

In [ ]:
df = pd.DataFrame(dataset['test'])

print("First few entries in the dataset:")
print(df.head(10))

I see that review with index 4 has html tags, so let's take a closser look at it 

In [ ]:
print(df.iloc[4]['review'])

Yes, the review has html tags, but it is not a big deal, because we can remove them easily if needed.

In [ ]:
print("Label:", df.iloc[4]['label'])

Interesting, while it is not obligatory to use "1" for positive and "0" for negative, it is a common practice. However this dataset uses "0" for positive and "1" for negative reviews.

# Statisctics and distributions of the data:

## Quality check

### Check for missing values

In [ ]:
df.info()

Gladly, there are no missing values in the dataset.

### Check for duplicates

In [ ]:
# Check for duplicate reviews
duplicate_count = df.duplicated(subset='review').sum()
print(f"\nNumber of duplicate reviews: {duplicate_count}")

# Remove duplicates if any
if duplicate_count > 0:
    df = df.drop_duplicates(subset='review').reset_index(drop=True)
    print("Duplicates have been removed.")

There are 16 duplicates in the dataset. While it is not a big deal, I will remove them, since they can skew analysis, models training and evaluation of the results.

In [ ]:
df = df.drop_duplicates()
df.shape

### Check class distribution

In [ ]:
print("Class distribution:")
print(df['label'].value_counts())

# Visualize class distribution
sns.countplot(x='label', data=df)
plt.title('Sentiment Class Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

The dataset is balanced, so we don't need to worry about class imbalance.

### Analyzing Review Lengths

In [ ]:
# Calculate review lengths (number of words)
df['review_length'] = df['review'].apply(lambda x: len(x.split()))

# Basic statistics of review lengths
print("\nReview Length Statistics:")
print(df['review_length'].describe())

# Visualize review length distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['review_length'], bins=50, kde=True)
plt.title('Review Length Distribution')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Boxplot of review length by sentiment
sns.boxplot(x='label', y='review_length', data=df)
plt.title('Review Length by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Number of Words')
os.makedirs('../results/eda', exist_ok=True)
plt.savefig('../results/eda/review_length_by_sentiment.png', bbox_inches='tight')
plt.show()

There is no big difference in review lengths by sentiment, so we don't need to worry about it while splitting.

# Lexical Analysis (Most Common Words)

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    """
    Function to preprocess text data
    """
    
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Apply preprocessing
df['tokens'] = df['review'].apply(preprocess_text)

In [ ]:
type(df['label'][0])

# Most Common Words in Positive and Negative Reviews

In [ ]:
# Separate positive and negative reviews
positive_reviews = df[df['label'] == 0]
negative_reviews = df[df['label'] == 1]

# Get all tokens for each class
positive_tokens = [token for tokens in positive_reviews['tokens'] for token in tokens]
negative_tokens = [token for tokens in negative_reviews['tokens'] for token in tokens]

# Get most common words
positive_counter = Counter(positive_tokens)
negative_counter = Counter(negative_tokens)

print("\nMost common words in positive reviews:")
print(positive_counter.most_common(20))

print("\nMost common words in negative reviews:")
print(negative_counter.most_common(20))


In [ ]:
N = 10
# Get the N most common words for positive and negative reviews
positive_common_words = positive_counter.most_common(N)
negative_common_words = negative_counter.most_common(N)

# Separate words and their counts for positive and negative reviews
positive_words, positive_counts = zip(*positive_common_words)
negative_words, negative_counts = zip(*negative_common_words)

# Plot for positive words
plt.figure(figsize=(7, 5))
plt.barh(positive_words, positive_counts, color='#a4c5ea')
plt.xlabel('Count')
plt.title('Most Common Words in Positive Reviews')
plt.gca().invert_yaxis()
plt.savefig('../results/eda/most_common_words_in_positive_reviews.png', bbox_inches='tight')
plt.show()

# Plot for negative words
plt.figure(figsize=(7, 5))
plt.barh(negative_words, negative_counts, color='#bca9e1')
plt.xlabel('Count')
plt.title('Most Common Words in Negative Reviews')
plt.gca().invert_yaxis()
plt.savefig('../results/eda/most_common_words_in_negative_reviews.png', bbox_inches='tight')
plt.show()


# Visualizing Word Clouds

In [ ]:
# Generate word cloud for positive reviews
positive_text = ' '.join(positive_tokens)
positive_wordcloud = WordCloud(width=800, height=400).generate(positive_text)

plt.figure(figsize=(15, 7.5))
plt.imshow(positive_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Positive Reviews')
plt.savefig('../results/eda/wordcloud_for_positive_tokens.png', bbox_inches='tight')
plt.show()

# Generate word cloud for negative reviews
negative_text = ' '.join(negative_tokens)
negative_wordcloud = WordCloud(width=800, height=400).generate(negative_text)

plt.figure(figsize=(15, 7.5))
plt.imshow(negative_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Negative Reviews')
plt.savefig('../results/eda/wordcloud_for_negative_tokens.png', bbox_inches='tight')
plt.show()


# Conclusions:

1. The dataset is balanced, so we don't need to worry about class imbalance.
2. There are no missing values in the dataset.
3. There are only 16 duplicates in the dataset, so it will not significantly affect any metrics.
4. There is no big difference in review lengths by sentiment, so we don't need to worry about it while splitting.
5. The most common words in positive and negative reviews are pretty similar, so we need to be careful
6. There are some unique words in positive and negative reviews, so it is possible to use encoding models to create embeddings for them and train a simple logreg model to predict sentiment - it can be a good baseline model.
7. There are some html tags in the reviews, so we need to remove them before training models.
